In [2]:
! pip install numpy
! pip install networkx
! pip install pgmpy
! pip install ipdb
! pip install yfinance
! pip install matplotlib
! pip install arviz
! pip install pymc
! pip install bambi
! pip install pyreadline



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import networkx
import pickle
import numpy as np
from stocks import Stocks, Ticker
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pickle
import matplotlib.pyplot as plt
from pprint import pprint
from stocks import Stocks, Ticker
import bambi as bmb
import xarray as xr
from StockEngine import StockEngine
from tqdm import tqdm

%config IPCompleter.greedy=True


In [2]:
with open("adj_mat.pkl", "rb") as f:
    G = pickle.load(f)
    
G.remove_node('DVA')
G.remove_node('ULTA')

tickers = list(G.nodes())

# UNCOMMENT IN FUTURE
# tickers = tickers[:5]

ticker_to_index = {}
for i, ticker in enumerate(tickers):
    ticker_to_index[ticker] = i
print(tickers)
print(ticker_to_index)
num_tickers = len(tickers)
corr_len = num_tickers * (num_tickers - 1) // 2

start_date = "2010-01-01"
end_date = "2021-01-01"

engine = StockEngine(tickers, start_date=start_date, end_date=end_date)

['CTSH', 'CI', 'CMS', 'ROST', 'JNJ', 'PFG', 'PAYX', 'AON', 'FIS', 'DXC', 'ROK', 'AMZN', 'ISRG', 'PNR', 'GWW', 'LNT', 'CE', 'NKE', 'MCO', 'AEP', 'TPR', 'UNH', 'NVR', 'XEL', 'TTWO', 'CMG', 'GIS', 'WST', 'CAT', 'XRAY', 'ANSS', 'QCOM', 'INTC', 'LIN', 'MU', 'GPC', 'ITW', 'EL', 'DXCM', 'COO', 'PKG', 'AMD', 'ETN', 'MHK', 'IEX', 'HON', 'MKC', 'MO', 'BIO', 'DIS', 'NFLX', 'RL', 'CSX', 'T', 'WM', 'TER', 'CRL', 'CL', 'JKHY', 'USB', 'COF', 'TFC', 'GS', 'AVB', 'UDR', 'ESS', 'HIG', 'PEAK', 'SYY', 'WAT', 'BWA', 'NEE', 'PPL', 'LLY', 'INCY', 'LVS', 'COST', 'XOM', 'SLB', 'DVN', 'WMB', 'PXD', 'EOG', 'PSA', 'TSCO', 'TAP']
{'CTSH': 0, 'CI': 1, 'CMS': 2, 'ROST': 3, 'JNJ': 4, 'PFG': 5, 'PAYX': 6, 'AON': 7, 'FIS': 8, 'DXC': 9, 'ROK': 10, 'AMZN': 11, 'ISRG': 12, 'PNR': 13, 'GWW': 14, 'LNT': 15, 'CE': 16, 'NKE': 17, 'MCO': 18, 'AEP': 19, 'TPR': 20, 'UNH': 21, 'NVR': 22, 'XEL': 23, 'TTWO': 24, 'CMG': 25, 'GIS': 26, 'WST': 27, 'CAT': 28, 'XRAY': 29, 'ANSS': 30, 'QCOM': 31, 'INTC': 32, 'LIN': 33, 'MU': 34, 'GPC': 3

100%|███████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 508.64it/s]


In [9]:
x_models = {}
x_model_pairs = []

# for ticker in tickers:
#     rv_models[ticker] = model

for i in range(len(tickers)):
    for j in range(i + 1, len(tickers)):
        ticker_i = tickers[i]
        ticker_j = tickers[j]
        model = pm.Model()
        model = pm.Model(name=f"x_{ticker_i}_{ticker_j}")
        if ticker_i not in x_models:
            x_models[ticker_i] = {}
            
        x_models[ticker_i][ticker_j] = model
        x_model_pairs.append((ticker_i, ticker_j))

In [10]:
import datetime

num_prev_trading_days = 126
months_to_evaluate = 12 * 10 # * 10
evaluation_year_month_start = "2018-01"

In [11]:
num_draws = 500
num_tune = 200
num_chains = 1

In [12]:
import ipdb

def train_volatilities(ticker, rv_observed, rv_day, rv_week, rv_month):
#     model = pm.Model(name=f"rv_{ticker}")
    model = pm.Model()
    with model:
        neighbors = list(G.adj[ticker].keys())
        num_neighbors_and_self = len(neighbors) + 1
        sigma = pm.HalfCauchy("sigma", beta=3)
        alpha = pm.HalfNormal("alpha", sigma=1)
#         alpha = pm.HalfNormal("alpha", mu=0, sigma=1)
        
        # Take self to be the first
        betas_day = pm.HalfNormal("betas_day", sigma=1, shape=num_neighbors_and_self)
        betas_week = pm.HalfNormal("betas_week", sigma=1, shape=num_neighbors_and_self)
        betas_month = pm.HalfNormal("betas_month", sigma=1, shape=num_neighbors_and_self)
#         betas_day = pm.HalfNormal("betas_day", mu=1/3, sigma=1, shape=num_neighbors_and_self)
#         betas_week = pm.HalfNormal("betas_week", mu=1/3, sigma=1, shape=num_neighbors_and_self)
#         betas_month = pm.HalfNormal("betas_month", mu=1/3, sigma=1, shape=num_neighbors_and_self)
    
        # Add myself
        ticker_index = ticker_to_index[ticker]
        pred = betas_day[0] * rv_day[:, ticker_index]
        pred += betas_week[0] * rv_week[:, ticker_index]
        pred += betas_month[0] * rv_month[:, ticker_index]
        
        pred += alpha
        
        for i, neighbor_ticker in enumerate(neighbors):
            neighbor_ticker_index = ticker_to_index[neighbor_ticker]
            pred += betas_day[i+1] * rv_day[:, neighbor_ticker_index]
            pred += betas_week[i+1] * rv_week[:, neighbor_ticker_index]
            pred += betas_month[i+1] * rv_month[:, neighbor_ticker_index]
            
        likelihood = pm.Normal(f"predicted_RV_{ticker}", mu = pred, sigma=sigma, observed=rv_observed[:, ticker_index])
        
        idata = pm.sample(draws=num_draws, tune=num_tune, chains=num_chains)
        
        return idata

In [13]:
def convert_to_lower_triangular(A, dimension):
    C = np.zeros((A.shape[0], dimension, dimension))
    B = np.zeros((dimension, dimension))
    idx = np.tril_indices_from(B, -1)
    
    for i in range(len(A)):
        C[i][idx] = A[i]
    return C

In [14]:
def train_correlations_distinct_coeff(ticker_1, ticker_2, x_observed, x_day, x_week, x_month):
    # model = pm.Model(name=f"rv_{ticker}")
    model = pm.Model()
    
    x_observed_mat = convert_to_lower_triangular(x_observed, num_tickers)
    x_day_mat = convert_to_lower_triangular(x_day, num_tickers)
    x_week_mat = convert_to_lower_triangular(x_week, num_tickers)
    x_month_mat = convert_to_lower_triangular(x_month, num_tickers)
    
    with model:
        sigma = pm.HalfCauchy("sigma", beta=3)
        alpha = pm.Normal("alpha", mu=0, sigma=1)
        
        betas_day = pm.Normal("betas_day", mu=1/3, sigma=1)
        betas_week = pm.Normal("betas_week", mu=1/3, sigma=1)
        betas_month = pm.Normal("betas_month", mu=1/3, sigma=1)
    
        ticker_1_index = ticker_to_index[ticker_1]
        ticker_2_index = ticker_to_index[ticker_2]
        pred = betas_day * x_day_mat[:, ticker_1_index, ticker_2_index]
        pred += betas_week * x_week_mat[:, ticker_1_index, ticker_2_index]
        pred += betas_month * x_month_mat[:, ticker_1_index, ticker_2_index]
        
        pred += alpha
        
        likelihood = pm.Normal(f"predicted_corr_{ticker}", mu = pred, sigma=sigma, observed=x_observed_mat[:, ticker_1_index, ticker_2_index])
        
        idata = pm.sample(draws=num_draws, tune=num_tune, chains=num_chains)
        
        return idata
    
    
def train_correlations_shared_coeff(x_observed, x_day, x_week, x_month):
    model = pm.Model()
    
    with model:
        sigma = pm.HalfCauchy("sigma", beta=3)
        alpha = pm.Normal("alpha", mu=0, sigma=1)
        
        betas_day = pm.Normal("betas_day", mu=1/3, sigma=1)
        betas_week = pm.Normal("betas_week", mu=1/3, sigma=1)
        betas_month = pm.Normal("betas_month", mu=1/3, sigma=1)
    
        pred = alpha
        pred += betas_day * x_day
        pred += betas_week * x_week
        pred += betas_month * x_month
        
        likelihood = pm.Normal(f"predicted_corr_{ticker}", mu = pred, sigma=sigma, observed=x_observed)
        
        idata = pm.sample(draws=num_draws, tune=num_tune, chains=num_chains)
        
        return idata

In [15]:
def predictions_vol_file(tday):
    return f"predictions/{tday}_vol.pkl"

def predictions_corr_file(tday):
    return f"predictions/{tday}_corr.pkl"

In [ ]:
dir = "predictions"

def predict_for_tday(tday):
    # Build our training samples
    RVt_all = np.zeros((num_prev_trading_days, num_tickers))
    RVt1_all = np.zeros((num_prev_trading_days, num_tickers))
    RVt25_all = np.zeros((num_prev_trading_days, num_tickers))
    RVt626_all = np.zeros((num_prev_trading_days, num_tickers))
    xt_all = np.zeros((num_prev_trading_days, corr_len))
    xt1_all = np.zeros((num_prev_trading_days, corr_len))
    xt25_all = np.zeros((num_prev_trading_days, corr_len))
    xt626_all = np.zeros((num_prev_trading_days, corr_len))
    for day in range(-num_prev_trading_days, 0):
        prev_tday = engine.get_shift_tday(tday, -day) 
        RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626 = engine.get_HAR_DRD_data(prev_tday)
        RVt_all[day + num_prev_trading_days] = RVt
        RVt1_all[day + num_prev_trading_days] = RVt1
        RVt25_all[day + num_prev_trading_days] = RVt25
        RVt626_all[day + num_prev_trading_days] = RVt626
        xt_all[day + num_prev_trading_days] = xt
        xt1_all[day + num_prev_trading_days] = xt1
        xt25_all[day + num_prev_trading_days] = xt25
        xt626_all[day + num_prev_trading_days] = xt626
        
    vol = {}
    # corr = {}
    
    corr = train_correlations_shared_coeff(xt_all, xt1_all, xt25_all, xt626_all)
#     for i in range(num_tickers):
#         for j in range(i + 1, num_tickers):
#             ticker_1 = tickers[i]
#             ticker_2 = tickers[j]
#             print("[Corr]", ticker_1, ticker_2)
#             idata_corr = train_correlations(ticker_1, ticker_2, xt_all, xt1_all, xt25_all, xt626_all)
#             corr[(ticker_1, ticker_2)] = idata_corr
    
    
    for ticker in tqdm(tickers):
        print("[Vol]", ticker)
        idata_vol = train_volatilities(ticker, RVt_all, RVt1_all, RVt25_all, RVt626_all)
        vol[ticker] = idata_vol
        
    with open(f"predictions/{tday}_vol.pkl", "wb") as f:
        pickle.dump(vol, f)
    with open(f"predictions/{tday}_corr.pkl", "wb") as f:
        pickle.dump(corr, f)
        
    return vol, corr 
        
print("Months to evaluate: ", months_to_evaluate)
USE_YEARLY = True
for month_diff in tqdm(range(months_to_evaluate)):
    if USE_YEARLY:
        year_month = engine.get_shift_month(evaluation_year_month_start, month_diff * 12)
    else:
        year_month = engine.get_shift_month(evaluation_year_month_start, month_diff)
        
    start_day = 1
    tday = year_month + "-{:02d}".format(start_day)
    while tday not in engine.trading_day_list:
        start_day += 1
        tday = year_month + "-{:02d}".format(start_day)
        
    print(f"Evaluating for {tday}")
    predict_for_tday(tday)
        
# predict_for_tday('2012-06-08')


Months to evaluate:  120


  0%|                                                                        | 0/120 [00:00<?, ?it/s]

Evaluating for 2018-01-02


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sigma, alpha, betas_day, betas_week, betas_month]


Sampling 1 chain for 200 tune and 500 draw iterations (200 + 500 draws total) took 122 seconds.
/home/fanpu/Documents/carnegie-mellon/10708/10-708-project/venv/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (500) than draws (1). Passed array should have shape (chains, draws, *shape)
  warnings.warn(

  0%|                                                                         | 0/86 [00:00<?, ?it/s]

...

[Vol] WMB


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sigma, alpha, betas_day, betas_week, betas_month]


Sampling 1 chain for 200 tune and 500 draw iterations (200 + 500 draws total) took 7 seconds.
/home/fanpu/Documents/carnegie-mellon/10708/10-708-project/venv/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (500) than draws (1). Passed array should have shape (chains, draws, *shape)
  warnings.warn(

 94%|████████████████████████████████████████████████████████████▎   | 81/86 [36:15<01:48, 21.64s/it]

[Vol] PXD


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


In [ ]:
1 + 1

In [79]:
engine.trading_day_list[1000:]

['2013-12-24',
 '2013-12-26',
 '2013-12-27',
 '2013-12-30',
 '2013-12-31',
 '2014-01-02',
 '2014-01-03',
 '2014-01-06',
 '2014-01-07',
 '2014-01-08',
 '2014-01-09',
 '2014-01-10',
 '2014-01-13',
 '2014-01-14',
 '2014-01-15',
 '2014-01-16',
 '2014-01-17',
 '2014-01-21',
 '2014-01-22',
 '2014-01-23',
 '2014-01-24',
 '2014-01-27',
 '2014-01-28',
 '2014-01-29',
 '2014-01-30',
 '2014-01-31',
 '2014-02-03',
 '2014-02-04',
 '2014-02-05',
 '2014-02-06',
 '2014-02-07',
 '2014-02-10',
 '2014-02-11',
 '2014-02-12',
 '2014-02-13',
 '2014-02-14',
 '2014-02-18',
 '2014-02-19',
 '2014-02-20',
 '2014-02-21',
 '2014-02-24',
 '2014-02-25',
 '2014-02-26',
 '2014-02-27',
 '2014-02-28',
 '2014-03-03',
 '2014-03-04',
 '2014-03-05',
 '2014-03-06',
 '2014-03-07',
 '2014-03-10',
 '2014-03-11',
 '2014-03-12',
 '2014-03-13',
 '2014-03-14',
 '2014-03-17',
 '2014-03-18',
 '2014-03-19',
 '2014-03-20',
 '2014-03-21',
 '2014-03-24',
 '2014-03-25',
 '2014-03-26',
 '2014-03-27',
 '2014-03-28',
 '2014-03-31',
 '2014-04-

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr

from pymc import HalfCauchy, Model, Normal, sample
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")


size = 200
true_intercept = 1
true_slope = 2

x = np.linspace(0, 1, size)
# y = a + b*x
true_regression_line = true_intercept + true_slope * x
# add noise
y = true_regression_line + rng.normal(scale=0.5, size=size)

data = pd.DataFrame(dict(x=x, y=y))
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel="x", ylabel="y", title="Generated data and underlying model")
ax.plot(x, y, "x", label="sampled data")
ax.plot(x, true_regression_line, label="true regression line", lw=2.0)
plt.legend(loc=0);

In [10]:
idata = None
with Model() as model:  # model specifications in PyMC are wrapped in a with-statement
    # Define priors
    sigma = HalfCauchy("sigma", beta=10)
    intercept = Normal("Intercept", 0, sigma=20)
    slope = Normal("slope", 0, sigma=20)

    # Define likelihood
    likelihood = Normal("y", mu=intercept + slope * x, sigma=sigma, observed=y)

    # Inference!
    # draw 3000 posterior samples using NUTS sampling
    idata = sample(4)


Only 4 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, slope]


Sampling 4 chains for 1_000 tune and 4 draw iterations (4_000 + 16 draws total) took 1 seconds.
/tmp/ipykernel_130024/3114496352.py:13: UserWarning: The number of samples is too small to check convergence reliably.
  idata = sample(4)


In [25]:
idata["posterior"]['sigma'][0][-1].to_numpy()

array(0.51696728)